Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

# Latent Dirchilet Allocation (LDA) Models (Prepare)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a modoel available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Litterary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for litterary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similiar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelility of British women publishing novels during the early 19th century. 

Can we help close a litterary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning alogrithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chuncks

In [2]:
path = './data/austen-brontë-split'

In [3]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [4]:
import os

def gather_data(path): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [5]:
df = pd.DataFrame()
df['tokens'] = gather_data(path)

In [6]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

In [7]:
df['tokens'].head()

0    [woods, miss, woodhouse, like, friend, knightl...
1    [semicircle, passing, round, giving, farewells...
2    [known, infant, seen, grow, period, notice, ho...
3    [better, change, mind, assure, power, took, le...
4    [wonder, amusement, secretly, added, lord, ble...
Name: tokens, dtype: object

## Follow Along

### Text Preprocessing
**Challenge**: update the function `tokenize` with any technique you have learned so far this week. 

In [8]:
titles = [t[:-4] for t in os.listdir(path)]

In [9]:
titles[:5]

['Austen_Emma0136',
 'CBronte_Villette0156',
 'Austen_Emma0111',
 'Austen_Sense0018',
 'Austen_Emma0100']

In [10]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [11]:
tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

### Author DataFrame


In [12]:
df = pd.DataFrame(index=titles)

In [13]:
df.head()

""
Austen_Emma0136
CBronte_Villette0156
Austen_Emma0111
Austen_Sense0018
Austen_Emma0100


In [14]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [15]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [16]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our fucntion. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [17]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(str(text))
                yield tokens

In [18]:
streaming_data = doc_stream(path)

In [19]:
print(streaming_data) # just a generator

<generator object doc_stream at 0x7fa65141ee58>


### Gensim LDA Topic Modeling

In [21]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [22]:
id2word.token2id['england']

1462

In [23]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(163, 1), (1462, 3), (2929, 1), (8928, 1)]

In [24]:
import sys
sys.getsizeof(id2word)

56

In [25]:
len(id2word.keys())

22095

In [26]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=10, no_above=0.75)

In [27]:
len(id2word.keys())

4921

In [28]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [29]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [30]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [31]:
lda.print_topics()

[(0,
  '0.015*"lady" + 0.015*"collins" + 0.014*"sir" + 0.010*"catherine" + 0.010*"miss" + 0.009*"bennet" + 0.008*"elizabeth" + 0.007*"ladies" + 0.006*"young" + 0.006*"william"'),
 (1,
  '0.008*"long" + 0.007*"miss" + 0.006*"room" + 0.005*"lady" + 0.005*"saw" + 0.005*"day" + 0.005*"monsieur" + 0.004*"having" + 0.004*"great" + 0.003*"pupil"'),
 (2,
  '0.014*"papa" + 0.008*"like" + 0.008*"old" + 0.006*"polly" + 0.005*"room" + 0.005*"letter" + 0.005*"red" + 0.005*"madame" + 0.004*"saw" + 0.004*"hands"'),
 (3,
  '0.010*"elizabeth" + 0.010*"gardiner" + 0.005*"walk" + 0.005*"stream" + 0.005*"woods" + 0.005*"civility" + 0.004*"felt" + 0.004*"darcy" + 0.004*"hunsford" + 0.004*"scene"'),
 (4,
  '0.010*"hunsden" + 0.007*"monsieur" + 0.006*"know" + 0.006*"come" + 0.005*"english" + 0.005*"frances" + 0.004*"shall" + 0.004*"soon" + 0.004*"suppose" + 0.004*"tell"'),
 (5,
  '0.009*"rochester" + 0.008*"jane" + 0.008*"sir" + 0.007*"like" + 0.005*"room" + 0.005*"thought" + 0.005*"shall" + 0.005*"night" + 

In [35]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [36]:
topics = [' '.join(t[0:5]) for t in words]

In [39]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
lady collins sir catherine miss

------ Topic 1 ------
long miss room lady saw

------ Topic 2 ------
papa like old polly room

------ Topic 3 ------
elizabeth gardiner walk stream woods

------ Topic 4 ------
hunsden monsieur know come english

------ Topic 5 ------
rochester jane sir like room

------ Topic 6 ------
elinor edward lucy marianne dashwood

------ Topic 7 ------
like thought know bretton day

------ Topic 8 ------
madame monsieur like door house

------ Topic 9 ------
elizabeth darcy bingley jane bennet

------ Topic 10 ------
like know john st diana

------ Topic 11 ------
like heart madame night thought

------ Topic 12 ------
like know think night day

------ Topic 13 ------
marianne elinor sister soon know

------ Topic 14 ------
emma miss harriet weston think



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [40]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

/home/ubuntu/.conda/envs/U4-S1-NLP/lib/python3.7/site-packages/_pytest/mark/structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [41]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/home/ubuntu/.conda/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.049130  0.012861       1        1  17.345165
13    -0.104832  0.021063       2        1  16.106424
14    -0.096366  0.011718       3        1  15.871459
5      0.053153  0.034102       4        1  13.152085
8      0.115908 -0.007628       5        1  10.983162
9     -0.137038 -0.035791       6        1   9.730810
11     0.097374  0.052567       7        1   6.469080
6     -0.139175  0.104931       8        1   3.212873
10     0.037973  0.069313       9        1   2.988794
0     -0.107462 -0.107067      10        1   1.417974
1      0.042025 -0.068173      11        1   0.830003
4      0.010536  0.022718      12        1   0.769152
12     0.018890 -0.018281      13        1   0.731156
2      0.117410 -0.011840      14        1   0.294061
3      0.042474 -0.080493      15        1   0.097798, topic_info=     Category         Freq          Term        Total  loglift  logprob
2198  Default   646.000000     elizabeth   646.000000  30.0000  30.0000
728   Default   674.000000        elinor   674.000000  29.0000  29.0000
126   Default  1337.000000          like  1337.000000  28.0000  28.0000
62    Default   874.000000          emma   874.000000  27.0000  27.0000
140   Default  1549.000000          miss  1549.000000  26.0000  26.0000
762   Default   553.000000      marianne   553.000000  25.0000  25.0000
2192  Default   421.000000         darcy   421.000000  24.0000  24.0000
118   Default  1443.000000          know  1443.000000  23.0000  23.0000
798   Default   606.000000           sir   606.000000  22.0000  22.0000
1907  Default   617.000000          lady   617.000000  21.0000  21.0000
1885  Default   319.000000        edward   319.000000  20.0000  20.0000
2003  Default   379.000000          lucy   379.000000  19.0000  19.0000
210   Default  1151.000000       thought  1151.000000  18.0000  18.0000
178   Default   892.000000          room   892.000000  17.0000  17.0000
129   Default   940.000000          long   940.000000  16.0000  16.0000
132   Default   408.000000        madame   408.000000  15.0000  15.0000
142   Default   669.000000        mother   669.000000  14.0000  14.0000
2337  Default   323.000000        bennet   323.000000  13.0000  13.0000
642   Default   777.000000           saw   777.000000  12.0000  12.0000
578   Default   529.000000       harriet   529.000000  11.0000  11.0000
209   Default  1326.000000         think  1326.000000  10.0000  10.0000
755   Default   628.000000          john   628.000000   9.0000   9.0000
620   Default   483.000000           old   483.000000   8.0000   8.0000
114   Default   957.000000          jane   957.000000   7.0000   7.0000
717   Default   250.000000      dashwood   250.000000   6.0000   6.0000
799   Default   584.000000        sister   584.000000   5.0000   5.0000
466   Default   927.000000          soon   927.000000   4.0000   4.0000
1040  Default   168.000000          papa   168.000000   3.0000   3.0000
43    Default  1060.000000           day  1060.000000   2.0000   2.0000
602   Default   644.000000          look   644.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
3272  Topic15     0.369524        firmly    16.386086   3.1380  -6.6319
460   Topic15     0.869962          slow    39.482552   3.1149  -5.7757
3524  Topic15     0.921332       crossed    47.294876   2.9917  -5.7183
1960  Topic15     1.313521      civility    77.461464   2.8529  -5.3636
584   Topic15     0.801392        humble    40.867203   2.9983  -5.8578
374   Topic15     0.904738        longed    53.109558   2.8575  -5.7365
3751  Topic15     0.918427      opposite    56.975494   2.8023  -5.7214
3963  Topic15     0.774626    companions    43.372665   2.9048  -5.8917
2198  Topic15     2.889766     elizabeth   646.191101   1.5201  -4.5752
1851  Topic15     0.839549          spot    58.695538   2.6828  -5.8112
3266  Topic15  

### Overall Model / Documents

In [49]:
lda[corpus[0]]

[(4, 0.077492304), (14, 0.9197196)]

In [50]:
distro = [lda[d] for d in corpus]

In [51]:
distro[0]

[(4, 0.07749483), (14, 0.9197172)]

In [52]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [53]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [54]:
df.head()

,lady collins sir catherine miss,long miss room lady saw,papa like old polly room,elizabeth gardiner walk stream woods,hunsden monsieur know come english,rochester jane sir like room,elinor edward lucy marianne dashwood,like thought know bretton day,madame monsieur like door house,elizabeth darcy bingley jane bennet,like know john st diana,like heart madame night thought,like know think night day,marianne elinor sister soon know,emma miss harriet weston think,author
Austen_Emma0136,0.0,0.0,0.0,0.0,0.077483,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.919729,Austen
CBronte_Villette0156,0.0,0.0,0.0,0.0,0.000000,0.075107,0.0,0.1412,0.450669,0.0,0.0,0.331137,0.0,0.000000,0.000000,CBronte
Austen_Emma0111,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.523379,0.474225,Austen
Austen_Sense0018,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.977330,0.020281,Austen
Austen_Emma0100,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.997278,Austen


In [55]:
df.groupby('author').mean()

,lady collins sir catherine miss,long miss room lady saw,papa like old polly room,elizabeth gardiner walk stream woods,hunsden monsieur know come english,rochester jane sir like room,elinor edward lucy marianne dashwood,like thought know bretton day,madame monsieur like door house,elizabeth darcy bingley jane bennet,like know john st diana,like heart madame night thought,like know think night day,marianne elinor sister soon know,emma miss harriet weston think
author,,,,,,,,,,,,,,,
Austen,0.027104,0.001782,0.000000,0.001885,0.002407,0.005711,0.066386,0.003300,0.004173,0.220833,0.000801,0.000892,0.002885,0.325976,0.333043
CBronte,0.002905,0.013133,0.004718,0.000000,0.012465,0.250860,0.001419,0.309355,0.189863,0.003448,0.057661,0.113747,0.010816,0.019004,0.007478


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [56]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, path, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = list(doc_stream(path))
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = doc_stream(path)
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [57]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        path=path, 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=40)

In [58]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [59]:
topic_coherence.head()

,coherence_score,num_topics,pass
0,-0.867331,2,0
1,-0.737585,8,0
2,-0.850439,14,0
3,-0.939246,20,0
4,-0.919345,26,0


In [60]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

/home/ubuntu/.conda/envs/U4-S1-NLP/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [61]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

NameError: name 'x' is not defined

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [ ]:
test = "Ned asked me a question about England today."

In [ ]:
doc = nlp(test)

for token in doc:
    print(token.text, token.lemma_, token.pos_)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
def doc_stream(path):
    for f in os.listdir(path):
        with open(os.path.join(path,f)) as t:
            text = t.read().strip('\n')
            yield text

def get_people(docstream):
    
    ppl = []
    
    for d in docstream:
        
        doc = nlp(d)
        
        for ent in doc.ents:
            
            if ent.label_ == "PERSON":
                ppl.append(ent.lemma_)
                
    return set(ppl)

In [ ]:
people = get_people(doc_stream(path))

In [ ]:
doc = nlp(next(doc_stream(path)))

In [ ]:
doc.ents[0].lemma_

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)